In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
path = "C:/Users/jacob.carey/Desktop/Projects/ConsentRateProjects/user_level_consentv1/data_prep/mass_athena/mass_athena_ba_merged_v2.csv"
columns = [
    #'companyid', 
    #'noticeid', 
    #'host', 
    #'companycountryid',
    'consentgiven', 
    #'usercountry', 
    #'actiontypeid', 
    'consenttypeid',
    'accesstypeid', 
    'os', 
    'browser', 
    #'usercorrelationkey', 
    'clearlanguage',
    #'CompanyName', 
    #'industry', 
    'onbrand', 
    #'snid'
]
dtypes = {
    #'companyid': np.int8, 
    #'noticeid': np.int8, 
    #'host', 
    #'companycountryid': np.int8,
    'consentgiven': np.int8, 
    #'usercountry', 
    #'actiontypeid': np.int8, 
    'consenttypeid': np.int8,
    'accesstypeid': np.int8, 
    #'os', 
    #'browser', 
    #'usercorrelationkey', 
    'clearlanguage': np.int8,
    #'CompanyName', 
    #'industry', 
    'onbrand': np.int8, 
    #'snid'
}

In [ ]:
df = pd.read_csv(path)

In [ ]:
df = df[df.consenttypeid != 0]

In [ ]:
df = df[df.consenttypeid != 1]
df = df[df.accesstypeid != 0]

In [ ]:
df['accesstypeid'].unique()

In [ ]:
df['id'] = df.index

In [ ]:
df.to_csv("C:/Users/jacob.carey/Desktop/Projects/ConsentRateProjects/user_level_consentv1/data_prep/mass_athena/mass_athena_ba_merged_v3.csv")

In [ ]:
outcome_col = 'consentgiven'
id_col = 'id'
feature_cols = [
    #'companyid', 
    #'noticeid', 
    #'host', 
    #'companycountryid',
    #'consentgiven', 
    #'usercountry', 
    #'actiontypeid', 
    'consenttypeid',
    'accesstypeid', 
    #'os', 
    #'browser', 
    #'usercorrelationkey', 
    'clearlanguage',
    #'CompanyName', 
    #'industry', 
    'onbrand', 
    #'snid'  
]

In [ ]:
# global counts
total_count = len(df.index)
total_positives = len(df[(df[outcome_col] == True)])

# An empty list to hold the count results
count_list = list()

# Iterate over each feature to collect individual counts per value
for feature in feature_cols:
    counts_df = pd.DataFrame(df.groupby([feature,])[id_col].count()).nlargest(20,[id_col])
    counts_df.rename({id_col: 'count'}, axis='columns', inplace=True)
    positives_df = pd.DataFrame(df[(df[outcome_col] == True)].groupby([feature,])[id_col].count())  # all values
    positives_df.rename({id_col: 'positives'}, axis='columns', inplace=True)
    # merge these dataframes
    merge_df = counts_df.merge(positives_df, left_index=True, right_index=True)
    # iterate over rows, building a dict per row, and append each to list of counts
    for this_row in merge_df.iterrows():
        this_dict = {
            'feature': feature,
            'value': str(this_row[0]),
            'total_count': total_count,
            'total_positives': total_positives,
            'count': this_row[1]['count'],
            'positives': this_row[1]['positives']
        }
        count_list.append(this_dict)

# Create a new dataframe from the aggregated list
lift_df = pd.DataFrame(count_list)

# Now let's add some calculations for probabilities and lift per row
lift_df['total_prob'] = lift_df['total_positives'] / lift_df['total_count']
lift_df['prob'] = lift_df['positives'] / lift_df['count']
lift_df['lift - more likely'] = lift_df['prob'] / lift_df['total_prob']
lift_df['lift - less likely'] = 1 / lift_df['lift - more likely']
lift_df['prct_total'] = lift_df['count'] / lift_df['total_count']

In [ ]:
date_time = datetime.today().strftime('%m-%d-%H%M')
output_filepath = "C:/Users/jacob.carey/Desktop/Projects/ConsentRateProjects/user_level_consentv1/data_prep/mass_athena/univariate/mass_athean_univariate_{0}.csv".format(date_time)
lift_df.to_csv(output_filepath, index=False)